# ETL Latin American Dataset of Presidential Elections and Campaigns - (LaDaPrEC)

This project shows a complete proccess of data extraction, transformation and load in *python*, applied to election data of Latin American countries. 

Main skills developed: 

- Creation of metrics and index in order to gain significant insights: creation of the Predominant Mobilization Index and degree of neutrality of the campaigns;
- Integration of databases in order to construct a comprehensive and coherent dataset;

In [51]:
import pandas as pd
import numpy as np

# Political Offer Variables - Evaluators and campaign Database - 'metabaseeclassificacao.csv'

In [32]:

# Extracting and merging the evaluators databases

gab = pd.read_excel("Bases finais/Classificação/Fichas de classificação/Gabrielle.xlsx")

gab = gab.drop(gab.columns[-1], axis=1)
gab = gab.astype({"ano_eleicao": "Int64", "tipo_apelo": "Int64", "emocao_apelo": "Int64"})
gab = gab.astype({"ano_eleicao": "category", "tipo_apelo": "category", "emocao_apelo": "category"})
gab.dropna(subset=["ano_eleicao"], inplace=True)

hel = pd.read_excel("Bases finais/Classificação/Fichas de classificação/Helena.xlsx")

hel = hel.astype({"ano_eleicao": "Int64", "tipo_apelo": "Int64", "emocao_apelo": "Int64", "musica":"Int64",
                  "tema":"Int64", "tempo_apelo_pais":"Int64", "tom_tempo_apelo":"Int64"})
hel = hel.astype({"ano_eleicao": "category", "tipo_apelo": "category", "emocao_apelo": "category", "musica":"category",
                  "tema":"category", "tempo_apelo_pais":"category", "tom_tempo_apelo":"category"})


xim = pd.read_excel("Bases finais/Classificação/Fichas de classificação/Ximena.xlsx")


xim = xim.astype({"ano_eleicao": "Int64", "tipo_apelo": "Int64", "emocao_apelo": "Int64", "musica":"Int64",
                  "tema":"Int64", "tempo_apelo_pais":"Int64", "tom_tempo_apelo":"Int64"})
xim = xim.astype({"ano_eleicao": "category", "tipo_apelo": "category", "emocao_apelo": "category", "musica":"category",
                  "tema":"category", "tempo_apelo_pais":"category", "tom_tempo_apelo":"category"})


gu = pd.read_excel("Bases Finais/Classificação/Fichas de classificação/Gustavo.xlsx", header=1)


gu = gu.astype({"ano_eleicao": "Int64", "tipo_apelo": "Int64", "emocao_apelo": "Int64", "musica":"Int64",
                  "tema":"Int64", "tempo_apelo_pais":"Int64", "tom_tempo_apelo":"Int64"})
gu = gu.astype({"ano_eleicao": "category", "tipo_apelo": "category", "emocao_apelo": "category", "musica":"category",
                  "tema":"category", "tempo_apelo_pais":"category", "tom_tempo_apelo":"category"})


comp = pd.concat([gab, hel, xim, gu], ignore_index=True)
comp = comp.astype({"tipo_apelo" : "Int64"})
comp = comp.astype({"tipo_apelo" : "category", "emocao_apelo":"object"})
#comp.to_csv('Bases finais/classificacao.csv')

In [33]:
# Joining the evaluators database with the main campaign database LaDaPrEC (campaign database = metabase.xlsx; comp = classificacao.csv)

met = pd.read_excel("Bases finais/metabase.xlsx", dtype = {"Orientacao":"category", "Candidato_minoria":"category",
                                                          "Incumbente":"category", "Situacao_oposicao":"category",
                                                          "candidato_2turno":"category", "candidato":"category"})

met["Tempo total (HH:MM:SS)"] = met["Tempo total (HH:MM:SS)"].dt.time


classi = pd.read_csv("Bases finais/classificacao.csv", dtype={"pais":"category", "candidato":"category",
                                                              "tipo_apelo":"category", "emocao_apelo":"category"})

classi["pais"] = classi["pais"].map(lambda x: "México" if x == "Méxio" else x)
classi["candidato"] = classi["candidato"].map(lambda x: "Nestor Kirchner" if x == "Nestor Kirchner a" else x)
       
combi = pd.merge(met, classi, on=["pais", "ano_eleicao", "candidato"], how="outer")

# Cleaning and recoding variables

combi.musica= np.where(combi.musica=='0', '3', combi.musica)

combi = combi.astype({'emocao_apelo':'object'})
combi = combi.replace({'musica' : {'NA ': np.nan},
                       'tema' : {'NA ': np.nan}, 
                       'tempo_apelo_pais' : {'NA ': np.nan, '5': np.nan, ',' : np.nan},
                       'tom_tempo_apelo' : {',' : np.nan, '10' : np.nan}})
combi = combi.replace( {'emocao_apelo' : np.nan, 
                        'musica': np.nan, 
                        'tema': np.nan,  
                        'tempo_apelo_pais': np.nan, 
                        'tom_tempo_apelo': np.nan}, 99)

combi = combi.astype({'emocao_apelo':'int64',
                      'musica':'int64',
                      'tema':'int64',
                      'tempo_apelo_pais':'int64',
                      'tom_tempo_apelo':'int64'})

combi = combi.astype({'emocao_apelo':'category',
                      'musica':'category',
                      'tema':'category',
                      'tempo_apelo_pais':'category',
                      'tom_tempo_apelo':'category'})

combi = combi.replace( {'emocao_apelo' : 99,
                        'musica': 99, 
                        'tema': 99,  
                        'tempo_apelo_pais': 99, 
                        'tom_tempo_apelo': 99}, np.nan)


#combi.to_csv('Bases finais/metabaseeclassificacao.csv', index=False)

print(combi.shape)
combi.head()

(48994, 26)


/var/folders/5t/m2j8g3ms4714qrywsn5fg_bc0000gn/T/ipykernel_54650/2879882728.py:45: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  combi = combi.replace( {'emocao_apelo' : 99,


,pais,ano_eleicao,candidato,Orientacao,Candidato_minoria,Incumbente,Situacao_oposicao,Partidos,Votos_1T,Votos_2T,...,candidato_2turno,Link eleição,codif,nome_video,tipo_apelo,emocao_apelo,musica,tema,tempo_apelo_pais,tom_tempo_apelo
0,Argentina,1983,Oscar Alende,2,0,0,2,PI,2.33,NaN,...,NaN,https://es.wikipedia.org/wiki/Elecciones_presi...,Gabrielle,Bancarrota,-1,2,3,0,1,3
1,Argentina,1983,Oscar Alende,2,0,0,2,PI,2.33,NaN,...,NaN,https://es.wikipedia.org/wiki/Elecciones_presi...,Gabrielle,Bancarrota,-1,2,3,0,0,0
2,Argentina,1983,Oscar Alende,2,0,0,2,PI,2.33,NaN,...,NaN,https://es.wikipedia.org/wiki/Elecciones_presi...,Gabrielle,Bancarrota,-1,2,3,0,0,0
3,Argentina,1983,Oscar Alende,2,0,0,2,PI,2.33,NaN,...,NaN,https://es.wikipedia.org/wiki/Elecciones_presi...,Gabrielle,Unidad Nacional,-1,3,3,0,0,0
4,Argentina,1983,Oscar Alende,2,0,0,2,PI,2.33,NaN,...,NaN,https://es.wikipedia.org/wiki/Elecciones_presi...,Gabrielle,Unidad Nacional,-1,3,3,0,0,0


In [36]:

## Collecting Anual Per Capita GDP Variation, DHI, Consumer Price Index (CPI), Unemployment rates - 'metagdpidh.csv'

# Inserting GDP Data

met = pd.read_csv('Bases finais/metabaseeclassificacao.csv', dtype={'tom_tempo_apelo':'category', 'tipo_apelo': 'Int64',
                                                                 'N_video': 'Int64', 'emocao_apelo':'category', 'musica':'category',
                                                                 'tempo_apelo_pais':'category', 'tema':'category'})

gdp = pd.read_csv("~/Downloads/Variáveis estruturais/GDP_Growth_per_capita/metadata.csv", encoding="latin-1")
gdp = gdp[gdp['Series Code']=='NY.GDP.PCAP.KD.ZG'].reset_index(drop=True)
gdp = gdp.drop(columns={'Series Name', 'Series Code', 'Country Code'}) 

column_names = gdp.iloc[:, 1:41].columns.tolist()
anos = list(range(1983,2023))

columns = {'Country Name':'pais'}
for col, ano in zip(column_names, anos):
    columns[col] = str(ano)

gdp.rename(columns=columns, inplace=True)


# Melting dataframe GDP to join with LaDaPrEC
GDP_melted = pd.melt(gdp, id_vars=["pais"], var_name="ano", value_name="var_anual_PIBpercapita")


GDP_melted = GDP_melted.rename(columns = {"Country Name" : "pais", "ano": "ano_eleicao"})
GDP_melted["pais"] = GDP_melted["pais"].replace(['Brazil', 'Ecuador', 'Mexico', 'Uruguay', 'Nicaragua', 'Venezuela, RB'], 
                                                ['Brasil', 'Equador', 'México', 'Uruguai', 'Nicarágua', 'Venezuela'])

GDP_melted["ano_eleicao"] = GDP_melted["ano_eleicao"].astype(int)


combi2 = pd.merge(met, GDP_melted, on=["pais", "ano_eleicao"], how="left")
combi2.tail()
#combi2.to_csv('Bases finais/metaclassigdp.csv', index=False)

,pais,ano_eleicao,candidato,Orientacao,Candidato_minoria,Incumbente,Situacao_oposicao,Partidos,Votos_1T,Votos_2T,...,Link eleição,codif,nome_video,tipo_apelo,emocao_apelo,musica,tema,tempo_apelo_pais,tom_tempo_apelo,var_anual_PIBpercapita
48989,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,https://es.wikipedia.org/wiki/Elecciones_presi...,Ximena Chávez,Sigamos Juntos testimonio 22,1,2,2,7,1,1,0.0945791076821791
48990,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,https://es.wikipedia.org/wiki/Elecciones_presi...,Ximena Chávez,Unidos en un abrazo,0,2,2,7,2,1,0.0945791076821791
48991,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,https://es.wikipedia.org/wiki/Elecciones_presi...,Ximena Chávez,Unidos en un abrazo,1,2,2,4,1,1,0.0945791076821791
48992,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,https://es.wikipedia.org/wiki/Elecciones_presi...,Ximena Chávez,Unidos en un abrazo,1,2,2,1,1,1,0.0945791076821791
48993,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,https://es.wikipedia.org/wiki/Elecciones_presi...,Ximena Chávez,Unidos en un abrazo,0,2,2,0,1,1,0.0945791076821791


In [37]:
# Inserting HDI and other development index

metgdp = pd.read_csv('Bases finais/metaclassigdp.csv')
idh = pd.read_csv('~/Downloads/Variáveis estruturais/Banco IDH/IDHeoutros.csv')

idh

,iso3,country,hdicode,region,hdi_rank_2021,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
0,AFG,Afghanistan,Low,SA,180.0,0.273,0.279,0.287,0.297,0.292,...,1.860000,1.880000,1.660000,1.620000,1.660000,1.410000,1.320000,1.380000,1.380000,1.380000
1,AGO,Angola,Medium,SSA,148.0,NaN,NaN,NaN,NaN,NaN,...,4.090000,4.530000,3.970000,3.590000,2.790000,2.640000,2.280000,2.180000,2.180000,2.180000
2,ALB,Albania,High,ECA,67.0,0.647,0.629,0.614,0.617,0.624,...,12.440000,11.490000,13.140000,12.610000,14.390000,14.460000,12.850000,12.960000,12.960000,12.960000
3,AND,Andorra,Very High,NaN,40.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,United Arab Emirates,Very High,AS,26.0,0.728,0.739,0.742,0.748,0.755,...,49.560000,49.680000,55.490000,59.760000,64.950000,75.610000,65.970000,68.950000,68.950000,68.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,ZZG.ECA,Europe and Central Asia,NaN,NaN,NaN,0.664,0.661,0.654,0.652,0.647,...,13.107171,14.172082,14.189127,14.523162,13.791770,13.618078,13.914030,14.087293,14.091826,14.096719
202,ZZH.LAC,Latin America and the Caribbean,NaN,NaN,NaN,0.633,0.638,0.643,0.648,0.654,...,14.114108,14.493145,14.080904,14.146026,13.325144,13.604555,13.151315,13.296195,13.290914,13.284745
203,ZZI.SA,South Asia,NaN,NaN,NaN,0.442,0.447,0.453,0.457,0.462,...,4.699407,4.584935,4.486500,4.610546,4.624886,4.740263,5.000024,5.110230,5.106447,5.102134
204,ZZJ.SSA,Sub-Saharan Africa,NaN,NaN,NaN,0.407,0.410,0.410,0.411,0.413,...,3.437760,3.530479,3.491150,3.494860,3.415511,3.256156,3.145930,3.151076,3.146092,3.140740


In [38]:
#Creating fuction to correct numerical formats - Don't need to use anymore

#def adicionar_zero(valor):
#    try:
#        valor_convertido = float(valor)
#        if valor_convertido > 1:
#            return float("0." + str(valor_convertido))
#        else:
#            return valor
#    except ValueError:
        # Valor não pode ser convertido para float, assume que não deve ser modificado
#        return valor

# Applying function to all columns

#for coluna in idh.columns[5:37]:
#    idh[coluna] = idh[coluna].apply(adicionar_zero)

#Selectiong and standardazing column names

column_names = idh.iloc[:, 5:37].columns.tolist()
anos = list(range(1990,2022))

dic_colnames = {'country':'pais'}
for col, ano in zip(column_names, anos):
    dic_colnames[col] = str(ano)

idh.rename(columns=dic_colnames, inplace=True)
idh = idh.loc[:, ["pais", *idh.columns[5:37]]]

# Melting dataframe HDI to join with LaDaPrEC
HDI_melted = pd.melt(idh, id_vars=["pais"], var_name="ano_eleicao", value_name="HDI")


HDI_melted["pais"] = HDI_melted["pais"].replace(['Brazil', 'Ecuador', 'Mexico', 'Uruguay', 'Nicaragua', 'Venezuela, RB', 
                                                 'Venezuela (Bolivarian Republic of)'], 
                                                ['Brasil', 'Equador', 'México', 'Uruguai', 'Nicarágua', 'Venezuela', 
                                                 'Venezuela'])

HDI_melted["ano_eleicao"] = HDI_melted["ano_eleicao"].astype(int)


combi3 = pd.merge(metgdp, HDI_melted, on=["pais", "ano_eleicao"], how="left")
combi3.tail()
#combi3.to_csv('Bases finais/baseofertaeleitoral.csv', index=False)

,pais,ano_eleicao,candidato,Orientacao,Candidato_minoria,Incumbente,Situacao_oposicao,Partidos,Votos_1T,Votos_2T,...,codif,nome_video,tipo_apelo,emocao_apelo,musica,tema,tempo_apelo_pais,tom_tempo_apelo,var_anual_PIBpercapita,HDI
48989,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Ximena Chávez,Sigamos Juntos testimonio 22,1.0,2.0,2.0,7.0,1.0,1.0,0.094579,0.774
48990,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Ximena Chávez,Unidos en un abrazo,0.0,2.0,2.0,7.0,2.0,1.0,0.094579,0.774
48991,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Ximena Chávez,Unidos en un abrazo,1.0,2.0,2.0,4.0,1.0,1.0,0.094579,0.774
48992,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Ximena Chávez,Unidos en un abrazo,1.0,2.0,2.0,1.0,1.0,1.0,0.094579,0.774
48993,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Ximena Chávez,Unidos en un abrazo,0.0,2.0,2.0,0.0,1.0,1.0,0.094579,0.774


In [39]:
# Inserting GNI

gni = pd.read_csv('~/Downloads/Variáveis estruturais/GNI_Tamanho_economia/data.csv')
gni= gni[gni['Series Code']=='NY.GNP.MKTP.PP.CD'].reset_index(drop=True)
gni = gni.drop(columns={'Series Name', 'Series Code', 'Country Code'}) 

column_names = gni.iloc[:, 1:41].columns.tolist()
anos = list(range(1983,2023))

columns = {'Country Name':'pais'}
for col, ano in zip(column_names, anos):
    columns[col] = str(ano)

gni.rename(columns=columns, inplace=True)

# Melting dataframe to join with LaDaPrEC
GNI_melted = pd.melt(gni, id_vars=["pais"], var_name="ano_eleicao", value_name="GNI")


#GNI_melted = GNI_melted.rename(columns = {"Country Name" : "pais", "ano": "ano_eleicao"})
GNI_melted["pais"] = GNI_melted["pais"].replace(['Brazil', 'Ecuador', 'Mexico', 'Uruguay', 'Nicaragua', 'Venezuela, RB'], 
                                                ['Brasil', 'Equador', 'México', 'Uruguai', 'Nicarágua', 'Venezuela'])

GNI_melted["ano_eleicao"] = GNI_melted["ano_eleicao"].astype('Int64')


combi4 = pd.merge(combi3, GNI_melted, on=["pais", "ano_eleicao"], how="left")
combi4['GNI'] = combi4['GNI'].replace(['..'], [np.nan])
combi4.tail()

,pais,ano_eleicao,candidato,Orientacao,Candidato_minoria,Incumbente,Situacao_oposicao,Partidos,Votos_1T,Votos_2T,...,nome_video,tipo_apelo,emocao_apelo,musica,tema,tempo_apelo_pais,tom_tempo_apelo,var_anual_PIBpercapita,HDI,GNI
48989,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Sigamos Juntos testimonio 22,1.0,2.0,2.0,7.0,1.0,1.0,0.094579,0.774,NaN
48990,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Unidos en un abrazo,0.0,2.0,2.0,7.0,2.0,1.0,0.094579,0.774,NaN
48991,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Unidos en un abrazo,1.0,2.0,2.0,4.0,1.0,1.0,0.094579,0.774,NaN
48992,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Unidos en un abrazo,1.0,2.0,2.0,1.0,1.0,1.0,0.094579,0.774,NaN
48993,Venezuela,2013,Nicolas Maduro,1.0,0.0,0.0,1.0,PSUV,50.61,NaN,...,Unidos en un abrazo,0.0,2.0,2.0,0.0,1.0,1.0,0.094579,0.774,NaN


In [40]:
# Inserting Consumer Price Index - inflation data

inf = pd.read_csv('~/Downloads/Variáveis estruturais/inflation/data.csv')
inf= inf[inf['Series Code']=='FP.CPI.TOTL.ZG'].reset_index(drop=True)
inf = inf.drop(columns={'Series Name', 'Series Code', 'Country Code'}) 

column_names = inf.iloc[:, 1:41].columns.tolist()
anos = list(range(1983,2023))

columns = {'Country Name':'pais'}
for col, ano in zip(column_names, anos):
    columns[col] = str(ano)

inf.rename(columns=columns, inplace=True)

# Melting dataframe to join with LaDaPrEC
INF_melted = pd.melt(inf, id_vars=["pais"], var_name="ano", value_name="inflacao_CPI")


INF_melted = INF_melted.rename(columns = {"Country Name" : "pais", "ano": "ano_eleicao"})
INF_melted["pais"] = INF_melted["pais"].replace(['Brazil', 'Ecuador', 'Mexico', 'Uruguay', 'Nicaragua', 'Venezuela, RB'], 
                                                ['Brasil', 'Equador', 'México', 'Uruguai', 'Nicarágua', 'Venezuela'])

INF_melted["ano_eleicao"] = INF_melted["ano_eleicao"].astype(int)


combi5 = pd.merge(combi4, INF_melted, on=["pais", "ano_eleicao"], how="left")
combi5['inflacao_CPI'] = combi5['inflacao_CPI'].replace(['..'], [np.nan])

In [41]:
# Insertin GINI database

gini = pd.read_csv('~/Downloads/Variáveis estruturais/Gini world bank/data.csv')
gini= gini[gini['Series Code']=='SI.POV.GINI'].reset_index(drop=True)
gini = gini.drop(columns={'Series Name', 'Series Code', 'Country Code'}) 

column_names = gini.iloc[:, 1:41].columns.tolist()
anos = list(range(1983,2023))

columns = {'Country Name':'pais'}
for col, ano in zip(column_names, anos):
    columns[col] = str(ano)

gini.rename(columns=columns, inplace=True)

# Melting dataframe to join with LaDaPrEC
GINI_melted = pd.melt(gini, id_vars=["pais"], var_name="ano", value_name="Gini_index")


GINI_melted = GINI_melted.rename(columns = {"Country Name" : "pais", "ano": "ano_eleicao"})
GINI_melted["pais"] = GINI_melted["pais"].replace(['Brazil', 'Ecuador', 'Mexico', 'Uruguay', 'Nicaragua', 'Venezuela, RB'], 
                                                ['Brasil', 'Equador', 'México', 'Uruguai', 'Nicarágua', 'Venezuela'])

GINI_melted["ano_eleicao"] = GINI_melted["ano_eleicao"].astype(int)

combi6 = pd.merge(combi5, GINI_melted, on=["pais", "ano_eleicao"], how="left")
combi6['Gini_index'] = combi6['Gini_index'].replace(['..'], [np.nan])

In [42]:
# Inserting Unemplyment data

une = pd.read_excel('~/Downloads/Variáveis estruturais/API_SL (desemprego)/data.xlsx', header=1)
une = une.drop(columns={'Country Code', 'Indicator Name', 'Indicator Code'})

UNE_melted = pd.melt(une, id_vars=["Country Name"], var_name="ano", value_name="desemprego")


UNE_melted = UNE_melted.rename(columns = {"Country Name" : "pais", "ano": "ano_eleicao"})
UNE_melted["pais"] = UNE_melted["pais"].replace(['Brazil', 'Ecuador', 'Mexico', 'Uruguay', 'Nicaragua', 'Venezuela, RB'], 
                                                ['Brasil', 'Equador', 'México', 'Uruguai', 'Nicarágua', 'Venezuela'])

UNE_melted["ano_eleicao"] = UNE_melted["ano_eleicao"].astype(int)


combi7 = pd.merge(combi6, UNE_melted, on=["pais", "ano_eleicao"], how="left")

combi7 = combi7.astype({'tipo_apelo':'Int64', 'emocao_apelo':'Int64', 'musica':'Int64', 'tema':'Int64',
                        'tom_tempo_apelo':'Int64'}, errors='ignore')

combi7['desemprego'] = combi7['desemprego'].replace({7583.0:7.583, 8426.0:8.426, 11089.0:11.089, 3169.0: 3.169, 6615.0:6.615,
                                                       5851.0:5.851})


In [43]:
# Inserting campaigns number of appeals and number of electoral programs

tes1 = combi7.groupby(['pais', 'ano_eleicao', 'candidato']).size().reset_index()
tes1.rename(columns={0:'N_apelo'}, inplace=True)

ofun = pd.merge(combi7, tes1, how='left')

ofun.replace({'musica':'NA ', 'tom_tempo_apelo':','}, np.nan, inplace = True)
ofun.replace({'tom_tempo_apelo':10}, 1, inplace = True)

ofun.fillna({'musica': 99, 'tom_tempo_apelo':99}, inplace=True)

ofun = ofun.astype({'tipo_apelo':'Int64', 'musica':'int64', 'tom_tempo_apelo':'int64'})

ofun.replace({'musica':99, 'tom_tempo_apelo':99}, None, inplace = True)

ofun = ofun.astype({'musica':'category', 'tom_tempo_apelo':'category'})

ofun[(ofun['pais']=='Argentina') & (ofun['ano_eleicao']==1983) & (ofun['candidato']=='Oscar Alende')][['pais', 'ano_eleicao', 'candidato', 'nome_video', 'Peças_TV', 'N_apelo']]

,pais,ano_eleicao,candidato,nome_video,Peças_TV,N_apelo
0,Argentina,1983,Oscar Alende,Bancarrota,2.0,5
1,Argentina,1983,Oscar Alende,Bancarrota,2.0,5
2,Argentina,1983,Oscar Alende,Bancarrota,2.0,5
3,Argentina,1983,Oscar Alende,Unidad Nacional,2.0,5
4,Argentina,1983,Oscar Alende,Unidad Nacional,2.0,5


In [44]:
df = ofun[['pais', 'ano_eleicao', 'candidato', 'Votos_1T', 'Votos_2T']].drop_duplicates()


# Calculating the difference of votes of the 2 most voted parties
df1 = df.groupby(
    ['pais', 'ano_eleicao'])[
    ['Votos_1T',  'Votos_2T']].agg(
    ['max', lambda x: x.sort_values(ascending=False).iloc[1] if x.shape[0] >= 2 else None,
     lambda x: x.sort_values(ascending=False).max() - x.sort_values(ascending=False).iloc[1] if x.shape[0] >= 2 else None]).reset_index()

df1.columns=['pais', 'ano_eleicao', 'maxi_1T', 'maxi_1T2nd', 'difs_1T', 'maxi_2T', 'maxi_2T2nd', 'difs_2T']
df1['dif_votos'] = np.where(df1['difs_2T'].notna(), df1['difs_2T'], df1['difs_1T'])
fin = df1[['pais', 'ano_eleicao', 'difs_1T', 'difs_2T', 'dif_votos']]

combi9 = pd.merge(ofun, fin[['pais', 'ano_eleicao', 'dif_votos']], how='left')
combi9['polariz'] = np.where(combi9.dif_votos >= 5, 1, 0)
combi9 = combi9.astype({'pais':'category','tom_tempo_apelo':'category', 'emocao_apelo':'category', 'tema':'category',
                        'tempo_apelo_pais':'Int64', 'dif_votos':'category'})

print(combi9.shape)
combi9.head()

(48994, 35)


,pais,ano_eleicao,candidato,Orientacao,Candidato_minoria,Incumbente,Situacao_oposicao,Partidos,Votos_1T,Votos_2T,...,tom_tempo_apelo,var_anual_PIBpercapita,HDI,GNI,inflacao_CPI,Gini_index,desemprego,N_apelo,dif_votos,polariz
0,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,3,2.73513,NaN,NaN,NaN,NaN,NaN,5,11.59,1
1,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,0,2.73513,NaN,NaN,NaN,NaN,NaN,5,11.59,1
2,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,0,2.73513,NaN,NaN,NaN,NaN,NaN,5,11.59,1
3,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,0,2.73513,NaN,NaN,NaN,NaN,NaN,5,11.59,1
4,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,0,2.73513,NaN,NaN,NaN,NaN,NaN,5,11.59,1


In [ ]:

# Computing voter's main problem and equivalence with campaigns main theme excluding ambiguous categories


test = combi9.copy()
test['tema'] = test['tema'].cat.remove_categories([0, 10])


test['tema'] = test.tema.cat.rename_categories({0: 'Outros temas',
                                                1:"Economia e desenvolvimento",
                                 4:"Segurança e justiça",
                                 2:"Corrupção e transparência",
                                 8:"Educação e conhecimento",
                                 3:"Saúde e bem-estar", 
                                 5:"Política externa", 
                                 6:"Igualdade e inclusão", 
                                 9:"Meio ambiente e sustentabilidade",
                                 7:"Valores e tradição",
                                 10:"Múltiplos temas"})


af = test.groupby(['pais', 'ano_eleicao', 'candidato'])
af = af.apply(lambda x: x['tema'].value_counts().nlargest(1)).reset_index()

ab = met[['pais', 'ano_eleicao', 'candidato']]

af1 = pd.merge(af, ab, how='left', on=['pais', 'ano_eleicao', 'candidato']).drop_duplicates().reset_index(drop=True)

# Loading electoral demand base and making the interest variable comparable adding the same categories as the ofer database

dem = pd.read_csv('Bases finais/basedemandaeleitoral.csv', dtype = {'princ_prob':'category'})
dem['princ_prob'] = dem['princ_prob'].cat.remove_categories(['Não sabe/ Não respondeu', 'Múltiplos temas', 'Outros temas'])
dem['princ_prob'] = dem['princ_prob'].cat.rename_categories({
    'Corrupção, governança e transparência':'Corrupção e transparência',
    'Política externa e relações internacionais':'Política externa'})
dem['princ_prob'] = dem['princ_prob'].cat.add_categories('Valores e tradição')

av = dem.groupby(['pais', 'ano_eleicao']).apply(lambda x: x['princ_prob'].value_counts().nlargest(1).reset_index())

df = pd.merge(af1, av, how='left', on=['pais', 'ano_eleicao'])


df['match_probtema_maincats'] = np.where(df.tema == df.princ_prob, 1, 0)
df = df.rename(columns={'princ_prob':'princ_probelec_maincats', 'tema':'princ_tema_maincats'})

df = df[['pais', 'ano_eleicao', 'candidato', 'princ_tema_maincats', 'princ_probelec_maincats', 'match_probtema_maincats']]

In [49]:
ofer = pd.merge(combi9, df, how='left', on=['pais', 'ano_eleicao', 'candidato'])
ofer = ofer.rename(columns = {'tema':'temas_completo'})
ofer["pais"] = ofer["pais"].replace(['Panama', 'Colombia'], 
                                ['Panamá', 'Colômbia'])
#ofer.to_csv("Bases finais/baseofertaeleitoral.csv", index=False)
print(ofer.shape)
ofer.head()

(48994, 38)


,pais,ano_eleicao,candidato,Orientacao,Candidato_minoria,Incumbente,Situacao_oposicao,Partidos,Votos_1T,Votos_2T,...,GNI,inflacao_CPI,Gini_index,desemprego,N_apelo,dif_votos,polariz,princ_tema_maincats,princ_probelec_maincats,match_probtema_maincats
0,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,NaN,NaN,NaN,NaN,5,11.59,1,Economia e desenvolvimento,NaN,0
1,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,NaN,NaN,NaN,NaN,5,11.59,1,Economia e desenvolvimento,NaN,0
2,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,NaN,NaN,NaN,NaN,5,11.59,1,Economia e desenvolvimento,NaN,0
3,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,NaN,NaN,NaN,NaN,5,11.59,1,Economia e desenvolvimento,NaN,0
4,Argentina,1983,Oscar Alende,2.0,0.0,0.0,2.0,PI,2.33,NaN,...,NaN,NaN,NaN,NaN,5,11.59,1,Economia e desenvolvimento,NaN,0


In [ ]:

ofer = pd.read_csv("Bases finais/baseofertaeleitoral.csv", dtype={'pais':'category','tom_tempo_apelo':'category', 'tipo_apelo': 'Int64',
                                                                 'N_video': 'Int64', 'emocao_apelo':'category', 'musica':'category',
                                                                 'tempo_apelo_pais':'category', 'temas_completo':'category',
                                                                 'princ_tema_maincats':'category', 
                                                                 'princ_probelec_maincats':'category',
                                                                 'match_probtema_maincats':'Int64'})

# Political Demand Variables - Latinobarometer database - 'Latinobarometros.csv'

In [16]:

## Importing all the dataframes

anos = ['1995', '1998', '2000', '2002', '2004', '2005', '2006', '2008', '2009',
         '2010', '2011', '2013', '2015', '2016', '2017', '2020', '2023']

vars = {
    '1995' : ['pais', 'numero', 'wt', 'p31', 'p1', 'p13', 'p27i', 'p27j', 'p27m'], 
    '1998' : ['idenpa', 'numinves', 'pondera', 'sp52', 'sp1', 'sp8', 'sp38f', 'sp38g', 'sp38d'],
    '2000' : ['idenpa', 'numinves', 'wt', 'P52ST', 'P1ST', 'P12ST', 'P35ST_F', 'P35ST_G', 'P35ST_D'],
    '2002' : ['idenpa', 'numinves', 'wt', 'p64st', 'p2sta', 'p4st', 'p36std', 'p34stf', 'p34std'], 
    '2004' : ['idenpa', 'numinves', 'wt', 'p87st', 'p2st', 'p10st', 'p34stf', 'p34std', 'p34stc'],
    '2005' : ['idenpa', 'numinves', 'wt', 'p34st', 'p2st', 'p8st', 'p45sta', 'p47stb', 'p45stc'],
    '2006' : ['idenpa', 'numinves', 'wt', 'p47st', 'p2st', 'p10st', 'p24st_f', 'p24st_c', 'p24st_b'],
    '2008' : ['idenpa', 'numinves', 'wt', 'p56st', 'p4st', 'p9st', 'p28st_a', 'p28st_c', 'p31s_ta'],
    '2009' : ['idenpa', 'numinves', 'wt', 'p69st', 'p3st_a', 'p2st', 'p26st_a', 'p26st_c', 'p24st_a'],
    '2010' : ['idenpa', 'numinves', 'wt', 'P60ST', 'P3ST_A', 'P2ST', 'P20ST_A', 'P20ST_C', 'P18ST_A'],
    '2011' : ['idenpa', 'numinves', 'wt', 'P76ST', 'P3ST_A', 'P2ST', 'P22ST_A', 'P22ST_C', 'P20ST_A'],
    '2013' : ['idenpa', 'numinves', 'wt', 'P41ST', 'P3STGBS', 'P9STGBS', 'P26TGB_C', 'P26TGB_G', 'P26TGB_A'],
    '2015' : ['idenpa', 'numinves', 'wt', 'P27ST', 'P3STGBS', 'P9STGBS', 'P16ST_F', 'P19ST_C', 'P16ST_G'],
    '2016' : ['idenpa', 'numinves', 'wt', 'P17ST', 'P4STGBS', 'P3STGBS', 'P13STD', 'P13STG', 'P13STE'],
    '2017' : ['idenpa', 'numinves', 'wt', 'P19STC', 'P4STGBSC', 'P3STGBS', 'P14ST_D', 'P14ST_G', 'P14ST_E'],
    '2020' : ['idenpa', 'numinves', 'wt', 'p18st', 'p4stgbs', 'p3stgbs', 'p13st_d', 'p13st_g', 'p13st_e'],
    '2023' : ['idenpa', 'numinves', 'wt', 'P16ST', 'P5STGBS', 'P4STGBS', 'P13ST_D', 'P13ST_G', 'P13ST_I']
    }

bases_latin = {} 
for ano in anos:
    vars_dinamicas = locals()
    vars_dinamicas[f"base_{ano}"] = pd.read_stata(f"Bases finais/Bases latinobarometro/{ano}.dta", columns=vars[ano])
    vars_dinamicas[f"base_{ano}"].columns = ['pais', 'ano', 'peso', 'posi_pol', 'sit_econ',
                                                          'princ_prob', 'conf_cong', 'conf_part', 'conf_presi']
    bases_latin[f"base_{ano}"] = vars_dinamicas[f"base_{ano}"]

lbcomp = pd.concat(bases_latin.values(), ignore_index=True)

In [17]:
# Cleaning variables

##Creating and applying function to standardize and reorder categories 

#Confidence variables

def remap_category(value):
    if value in ['A lot', 'A lot of confidence', 'Mucha', 'Lot']:
        return "A lot"
    elif value in ["Algo", "Some confidence", 'Some', 'Some confidence']:
        return "Some"
    elif value in ["A little", "Little", 'Little confidence', 'Poca']:
        return "A little"
    elif value in ['Ninguna', "Some confidence", 'No confidence at all', 'No trust', 'Nothing']:
        return "No confidence at all"
    else:
        return "Don't know/No answer"
    
lbcomp['conf_cong'] = lbcomp['conf_cong'].map(remap_category)
lbcomp['conf_part'] = lbcomp['conf_part'].map(remap_category)
lbcomp['conf_presi'] = lbcomp['conf_presi'].map(remap_category)


lbcomp = lbcomp.astype({'conf_presi':'category', 'conf_cong':'category', 'conf_part':'category'})
lbcomp['conf_presi'] = lbcomp['conf_presi'].cat.reorder_categories(["A lot", "Some", "A little", "No confidence at all",
                                                                  "Don't know/No answer"])
lbcomp['conf_part'] = lbcomp['conf_part'].cat.reorder_categories(["A lot", "Some", "A little", "No confidence at all",
                                                                  "Don't know/No answer"])
lbcomp['conf_cong'] = lbcomp['conf_cong'].cat.reorder_categories(["A lot", "Some", "A little", "No confidence at all",
                                                                  "Don't know/No answer"])

In [18]:
#Political and economical variables

#Current economical situation

def remap_category_econ(value):
    if value in ['Muy buena', 'Very good']:
        return "Very Good"
    elif value in ['Good',]:
        return "Good"
    elif value in ['About average', 'Regular' ]:
        return "About Average"
    elif value in ['Bad', 'Mala']:
        return "Bad"
    elif value in ['Muy Mala', 'Very bad']:
        return "Very Bad"
    else:
        return "Don't know/No answer"
    
lbcomp['sit_econ'] = lbcomp['sit_econ'].map(remap_category_econ)
lbcomp['sit_econ'] = lbcomp['sit_econ'].astype('category')
lbcomp['sit_econ'] = lbcomp['sit_econ'].cat.reorder_categories(["Very Good", "Good", "About Average", "Bad", "Very Bad", 
                                                                "Don't know/No answer"])

In [19]:
#Political position

def remap_category_posi(value):
    if value in ['Right', 'Derecha', '10 RIGHT', '10 Right', '10. Right']:
        return "10"
    elif value in ['0. Left', '00 LEFT', '00 Left', 'Izquierda', 'Left',  ]:
        return "0"
    elif value in ['6', -6]:
        return "6"
    elif value in ['Does not answer', "Don't know", 'Don´t answer', 'Don´t know', 'DonÂ´t know', 'Ninguno', 'No answer',
 'No answer/Refused', 'No responde', 'No sabe', 'None']:
        return "Don't know/No answer"
    else:
        return value
    
lbcomp['posi_pol'] = lbcomp['posi_pol'].map(remap_category_posi)
lbcomp['posi_pol'] = lbcomp['posi_pol'].astype('category')
lbcomp['posi_pol'] = lbcomp['posi_pol'].cat.reorder_categories(["0", "1", "2", "3", "4", '5', '6', '7', '8', '9', '10', 
                                                                "Don't know/No answer"])

In [20]:

#Main country problem

def remap_category_prob(value):
    if value in ['All expensive', 'Bajos salarios', 'Deficient basic services (water, electricity, ...)',
                'Desabastecimiento/Falta de alimentos/Acaparamiento', 'DesocupaciÃ³n / desempleo', 
                'DistribuciÃ³n del ingreso, injusticia social', 'Economy/ Economic problems/Financial problems', 
                'Economy/ Economic problems/financial problems', 'Economy/economical problems/financial', 
                'Employment inestability', 'Employment instability', 'Everything is very expensive', 'Food, scarcity',
                'Gas, fuel, scarcity, (high prices)', 'Income distribution', 'Income inequality, social injustice',
                'Inestabilidad en el empleo', 'InflaciÃ³n / aumento de precios', 'Inflation/ price rises',
                'Inflation/ price rises / Economic Crisis', 'Inflation/ raise of prices', 'Inflation/rise in prices',
                'Instability in employment', 'La economÃ\xada/problemas econÃ³micos/financieros', 'Low Salaries',
                'Low salaries', 'Pobreza', 'Poverty', 'Poverty / Social Inequality', 'Shortages / Lack of food / Hoarding',
                'Unemployment', 'shortage of groceries /Hoarding/ food is missing']:
        return "Economia e desenvolvimento"
    elif value in ['A lot of dirt', 'CorrupciÃ³n', 'Corruption', 'Dirtiness, lack of cleanliness']:
        return "Corrupção, governança e transparência"
    elif value in ['Coronavirus pandemic/ Covid-19', 'Drug consumption / addiction',
                  'Drugs consumption /addiction', 'Health', 'Health Problems', 'Health issues', 'Health problems', 
                  'Pandemic/Coronavirus/Covid-19', 'Problemas de la salud']:
        return "Saúde e bem-estar"
    elif value in ['Crime', 'Crime / Public security', 'Crime /Public Security', 'Crime/public security',
                   'Delincuencia / seguridad pÃºblica', 'Delinquency / public security', 'Drug trafficking', 
                   'Guerrilla warfare', 'Judicial System/ justice', 'Narcotrafic', 'NarcotrÃ¡fico', 'Terrorism',
                   'Terrorism / Political violence / Guerrilla', 'Terrorism/ war', 'Terrorism/political violence/guerrilla',
                   'Terrorismo/guerrilla', 'Verbal abuse', 'Verbal violence', 'Violence / gangs', 'Violence, bands',
                   'Violence/gangs', 'Violencia/pandillas']:
        return "Segurança e justiça"
    elif value in ['Border conflicts', 'Border issues', 'Border problems', 'Conflicts with neighboring countries', 
                   'Foreigners', 'Foreing people', 'Immigrants', 'Problemas con paÃ\xadses vecinos', 
                   'Problemas limÃ\xadtrofes', 'Problems with neighbor countries']:
        return "Política externa e relações internacionais"
    elif value in ['DiscriminaciÃ³n racial', 'Discrimination by race', 'Domestic violence', 'Racial Discrimination',
                   'Racial discrimination']:
        return "Igualdade e inclusão"
    elif value in []:
        return "Valores e tradição"
    elif value in ['Education', 'Education problems', 'Problemas de la educaciÃ³n', ]:
        return "Educação e conhecimento"
    elif value in ['Calentamiento global', 'Environment', 'Environment problems', 'Environmental problems', 'Global heating', 
                   'Global warming', 'Pollution', 'Problemas del medio ambiente/contaminaciÃ³n', 'Contamination', 
                   'environmental issues']:
        return "Meio ambiente e sustentabilidade"
    elif value in ['Instability in employment / Immigration']:
        return "Múltiplos temas"
    elif value in ['Does not answer', "Don't know", 'DonÂ´t know', 'Ninguno', 'No answer', 'No answer/Refused', 'No problem',
                   'No problems', 'No responde', 'No sabe', 'None', 'Not asked', 'Not being able to be in my country']:
        return "Não sabe/ Não respondeu"
    else:
        return "Outros temas"
    
lbcomp['princ_prob'] = lbcomp['princ_prob'].map(remap_category_prob)
lbcomp['princ_prob'] = lbcomp['princ_prob'].astype('category')


In [21]:
lbcomp['princ_prob'] = lbcomp['princ_prob'].cat.reorder_categories(["Economia e desenvolvimento",
                                                                    "Corrupção, governança e transparência",
                                                                    "Saúde e bem-estar", "Segurança e justiça",
                                                                    "Política externa e relações internacionais", 
                                                                    "Igualdade e inclusão", 
                                                                    "Educação e conhecimento",
                                                                    "Meio ambiente e sustentabilidade",
                                                                    "Múltiplos temas", "Outros temas",
                                                                    "Não sabe/ Não respondeu"])

lbcomp.head()

,pais,ano,peso,posi_pol,sit_econ,princ_prob,conf_cong,conf_part,conf_presi
0,Chile,1995,0.90782,5,Good,Segurança e justiça,Some,A little,Some
1,Chile,1995,0.90782,9,About Average,Economia e desenvolvimento,Some,Some,A little
2,Chile,1995,0.90782,5,Bad,Economia e desenvolvimento,A little,A little,Some
3,Chile,1995,0.90782,0,About Average,Economia e desenvolvimento,A little,Some,Some
4,Chile,1995,0.90782,5,About Average,Economia e desenvolvimento,Some,A little,A little


In [22]:
# Filtering to the country and years matching the offer database

In [23]:
met = pd.read_csv('Bases finais/metabaseeclassificacao.csv', dtype={'tipo_apelo':'category', 'emocao_apelo':'category', 
                                                                    'tom_tempo_apelo':'category'})

a = met[['pais', 'ano_eleicao']]
a = a.drop_duplicates()
a.sort_values('ano_eleicao', inplace = True)
a = a.reset_index(drop=True)

a.head()

,pais,ano_eleicao
0,Argentina,1983
1,Argentina,1989
2,Argentina,1995
3,Argentina,1999
4,México,2000


In [29]:
lbcomp.sort_values('ano', inplace = True)
lbcomp['ano'] = lbcomp['ano'].astype('int64')

#Removing blanckspaces before and after country names and standardizing them
lbcomp['pais'] = lbcomp['pais'].str.strip()
lbcomp["pais"] = lbcomp["pais"].replace(['Brazil', 'Ecuador', 'Mexico', 'Uruguay', 'Nicaragua', 'Venezuela, RB', 
                                                 'Venezuela (Bolivarian Republic of)', 'MÃ©xico', 'PerÃº'], 
                                                ['Brasil', 'Equador', 'México', 'Uruguai', 'Nicarágua', 'Venezuela', 
                                                 'Venezuela', 'México', 'Peru'])

#Merging political offer database with demand database -1 year
ten2 = pd.merge_asof(lbcomp, a, tolerance = 1, left_on='ano', right_on='ano_eleicao',
                    by='pais', allow_exact_matches=False, direction='forward')
ten2 = ten2.dropna(subset=['ano_eleicao']).reset_index(drop=True)

#Joining vertically countries by the same year in the 2 databases and creating year of the survey and election variables

df = pd.concat([ten2, lbcomp[(lbcomp['pais']=='Argentina') & (lbcomp['ano']==1995)],
                lbcomp[(lbcomp['pais']=='Argentina') & (lbcomp['ano']==2015)],
                lbcomp[(lbcomp['pais']=='Chile') & (lbcomp['ano']==2013)],
                lbcomp[(lbcomp['pais']=='Equador') & (lbcomp['ano']==2013)],
                lbcomp[(lbcomp['pais']=='Honduras') & (lbcomp['ano']==2013)],
                lbcomp[(lbcomp['pais']=='México') & (lbcomp['ano']==2000)],
                 lbcomp[(lbcomp['pais']=='Venezuela') & (lbcomp['ano']==2013)]], ignore_index=True)

conditions = [
    (df['pais'] == 'Argentina') & (df['ano'] == 1995),
    (df['pais']=='Argentina') & (df['ano']==2015),
    (df['pais']=='Chile') & (df['ano']==2013),
    (df['pais']=='Equador') & (df['ano']==2013),
    (df['pais']=='Honduras') & (df['ano']==2013),
    (df['pais'] == 'México') & (df['ano'] == 2000),
    (df['pais']=='Venezuela') & (df['ano']==2013),
    
]

choices = [df['ano'], df['ano'], df['ano'], df['ano'], df['ano'], df['ano'], df['ano']]

df['ano_eleicao'] = np.select(conditions, choices, default=df['ano_eleicao'])

df = df.sort_values('ano').reset_index(drop=True)
df.insert(2, 'ano_eleicao', df.pop('ano_eleicao'))
df = df.astype({'ano_eleicao':'Int64'})
df.rename(columns={'ano':'ano_survey'}, inplace = True)
df["pais"] = df["pais"].replace(['Panama', 'Colombia'], 
                                ['Panamá', 'Colômbia'])
#df.to_csv('Bases finais/basedemandaeleitoral.csv', index=False)
print(df.shape)
df.pais.unique()

(43178, 10)


array(['Argentina', 'México', 'Chile', 'Equador', 'Brasil', 'Colômbia',
       'Uruguai', 'Panamá', 'Honduras', 'Costa Rica', 'Peru', 'Nicarágua',
       'Guatemala', 'Venezuela'], dtype=object)

In [5]:
dem = pd.read_csv("Bases finais/basedemandaeleitoral.csv", dtype={'pais':'category', 'sit_econ':'category','princ_prob':'category',
                                                                  'conf_cong':'category', 'conf_part':'category', 
                                                                  'conf_presi':'category', 'peso':'float64', 'ano_eleicao':'category'})

pais           category
ano_survey        int64
ano_eleicao       int64
peso             object
posi_pol         object
sit_econ       category
princ_prob     category
conf_cong      category
conf_part      category
conf_presi     category
dtype: object

# Agregated database ofestim

In [61]:

ofer = pd.read_csv("Bases finais/baseofertaeleitoral.csv", dtype={'tom_tempo_apelo':'category', 'tipo_apelo': 'Int64',
                                                                 'N_video': 'Int64', 'emocao_apelo':'category', 'musica':'category',
                                                                 'tempo_apelo_pais':'category', 'temas_completo':'category',
                                                                 'princ_tema_maincats':'category', 
                                                                 'princ_probelec_maincats':'category',
                                                                 'match_probtema_maincats':'Int64'})
# Variables related to elections and campaigns

strut = ofer.iloc[:, 0:18].columns.append(ofer.iloc[:, 26:].columns)

# Variables related to electoral appeals

elect = ofer.iloc[:, 19:26].columns

# Filtering campaings for appeals

print(ofer.shape, ofer1.shape)
a = ofer[['pais', 'ano_eleicao', 'candidato']].drop_duplicates().reset_index(drop=True)
a.shape[0]

(48994, 38) (44725, 38)


156

# Neutrality Degree (GN) and Predominant Mobilization Index (IMP) of presidential electoral campaigns

Those índex were created to estimate how much of the electoral appeals of the campaigns were neutral (i.e., not directly providing reasons to vote for them - GN), and among the active appeals, if they are predominantly symbolic or material (IMP).

In [63]:


elect_var = ['pais', 'ano_eleicao']
elect_var += list(elect)

b = ofer[strut]
b = b.drop_duplicates().reset_index(drop=True)

a = ofer.copy().dropna(subset = 'tipo_apelo')

# Calculating neutrality degree of electoral campaigns

a['apelos_ativos']=np.where(a['tipo_apelo']==0, 0, 1)
a['neutralidade_apelo']=np.where(a['tipo_apelo']==0, 1, 0)

a = a.replace({'tipo_apelo':99}, None).groupby(['pais', 'ano_eleicao', 'candidato']).agg({
    'tipo_apelo': 'mean', 'apelos_ativos':'mean', 'neutralidade_apelo':'mean'}).reset_index()

ofestim = pd.merge(a, b, how='left', on=['pais', 'ano_eleicao', 'candidato'])

# Calculating Predominant Mobilization Index (IMP) of electoral campaigns

ofestim['IMP'] = ofestim['tipo_apelo'] * ofestim['apelos_ativos']
ofestim.Orientacao = ofestim.Orientacao.replace([1.0, 2.0, 3.0], ['Esquerda', 'Centro', 'Direita'])
ofestim.Situacao_oposicao = ofestim.Situacao_oposicao.replace([1.0, 2.0, 3.0], ['Situação', 'Oposição', 'Independente'])

# Colors used in graphs

cores_personalizadas = {
    "Economia e desenvolvimento": "#FFC107",  
    "Segurança e justiça": "#1E88E5",  
    "Educação e conhecimento": "#38C636",  
    "Corrupção e transparência": "#51868A",  
    "Igualdade e inclusão": "#571AFD", 
    "Saúde e bem-estar": "#29FBA4",
    "Valores e tradição": "#5B0FB0", 
    "Meio ambiente": "#44BF3F",
    "Política externa": "#309A38",
}

print(ofestim.shape)
ofestim
#ofestim.to_csv("ofestimfull.csv", index=False)

(155, 34)


,pais,ano_eleicao,candidato,tipo_apelo,apelos_ativos,neutralidade_apelo,Orientacao,Candidato_minoria,Incumbente,Situacao_oposicao,...,inflacao_CPI,Gini_index,desemprego,N_apelo,dif_votos,polariz,princ_tema_maincats,princ_probelec_maincats,match_probtema_maincats,IMP
0,Argentina,1983,Oscar Alende,-1.0,1.000000,0.000000,Centro,0.0,0.0,Oposição,...,NaN,NaN,NaN,5,11.59,1,Economia e desenvolvimento,NaN,0,-1.0
1,Argentina,1983,Raul Alfonsín,0.314286,0.771429,0.228571,Esquerda,0.0,0.0,Oposição,...,NaN,NaN,NaN,38,11.59,1,Economia e desenvolvimento,NaN,0,0.242449
2,Argentina,1983,Ítalo Lúder,-0.257143,0.833333,0.166667,Esquerda,0.0,0.0,Oposição,...,NaN,NaN,NaN,37,11.59,1,Economia e desenvolvimento,NaN,0,-0.214286
3,Argentina,1989,Carlos Saúl Menem,0.283465,0.708661,0.291339,Direita,0.0,0.0,Oposição,...,NaN,NaN,NaN,127,10.41,1,Economia e desenvolvimento,NaN,0,0.20088
4,Argentina,1989,Eduardo Angeloz,-0.109091,0.618182,0.381818,Direita,0.0,0.0,Situação,...,NaN,NaN,NaN,110,10.41,1,Economia e desenvolvimento,NaN,0,-0.067438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Uruguai,2009,Pedro Bordaberry,0.053097,0.230088,0.769912,Direita,0.0,0.0,Oposição,...,7.062219,45.5,7.74,226,9.26,1,Economia e desenvolvimento,Economia e desenvolvimento,1,0.012217
151,Venezuela,2012,Henrique Capriles,0.137405,0.631043,0.368957,Direita,0.0,0.0,Oposição,...,21.068996,NaN,6.60,401,10.76,1,Economia e desenvolvimento,Segurança e justiça,0,0.086708
152,Venezuela,2012,Hugo Chavez,0.274336,0.769912,0.230088,Esquerda,0.0,1.0,Situação,...,21.068996,NaN,6.60,113,10.76,1,Economia e desenvolvimento,Segurança e justiça,0,0.211215
153,Venezuela,2013,Henrique Capriles,0.233438,0.649842,0.350158,Direita,0.0,0.0,Oposição,...,40.639428,NaN,7.54,317,1.49,0,Corrupção e transparência,Segurança e justiça,0,0.151698
